In [2]:
%pip install pymilvus==2.4.13



  Using cached pymilvus-2.4.13-py3-none-any.whl.metadata (5.7 kB)
Using cached pymilvus-2.4.13-py3-none-any.whl (218 kB)
  Attempting uninstall: pymilvus
    Found existing installation: pymilvus 2.5.3
    Uninstalling pymilvus-2.5.3:
      Successfully uninstalled pymilvus-2.5.3
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
from pymilvus import connections

try:
    connections.connect(alias="default", host="127.0.0.1", port="19530")
    print("Connection successful!")
except Exception as e:
    print(f"Connection failed: {e}")


Connection successful!


In [4]:
elastic_ip = '34.0.64.248:9200'
kibana_ip = '34.0.64.248:5601'
es_username = 'user'
es_password = 'knesset'
!pip install elasticsearch


[notice] A new release of pip is available: 24.0 -> 24.3.1
[notice] To update, run: python.exe -m pip install --upgrade pip


In [5]:
from elasticsearch import Elasticsearch

es = Elasticsearch(f'http://{elastic_ip}', basic_auth=(es_username, es_password), request_timeout=500)
data_q =[]
# Query definition
query = {
    "query": {
        "bool": {
            "must": [
                {"match": {"speaker_name": "מירי רגב"}}
            ],
            "filter": {
                "script": {
                    "script": {
                        "source": "doc['sentence_text.keyword'].size() > 0 && doc['sentence_text.keyword'].value.length() > 10"
                    }
                }
            }
        }
    }
}

# Initialize scroll
resp = es.search(index="all_features_sentences", body=query, scroll="2m", size=1000)

# Retrieve the scroll ID and first batch of hits
scroll_id = resp['_scroll_id']
hits = resp['hits']['hits']

total_hits = 0
while total_hits<4000:
    for hit in hits:
      data_q.append("%(sentence_text)s" % hit["_source"])
      #print("%(sentence_text)s" % hit["_source"])

    total_hits += len(hits)

    # Fetch the next batch
    resp = es.scroll(scroll_id=scroll_id, scroll="2m")
    scroll_id = resp['_scroll_id']
    hits = resp['hits']['hits']

print(f"Total results retrieved: {total_hits}")

# Clear the scroll to free resources
es.clear_scroll(scroll_id=scroll_id)


c:\Users\1\AppData\Local\Programs\Python\Python310\lib\site-packages\requests\__init__.py:102: RequestsDependencyWarning: urllib3 (1.26.7) or chardet (5.2.0)/charset_normalizer (2.0.9) doesn't match a supported version!
  warnings.warn("urllib3 ({}) or chardet ({})/charset_normalizer ({}) doesn't match a supported "
C:\Users\1\AppData\Local\Temp\ipykernel_16176\3351779918.py:24: DeprecationWarning: Received 'size' via a specific parameter in the presence of a 'body' parameter, which is deprecated and will be removed in a future version. Instead, use only 'body' or only specific parameters.
  resp = es.search(index="all_features_sentences", body=query, scroll="2m", size=1000)


Total results retrieved: 4000


ObjectApiResponse({'succeeded': True, 'num_freed': 1})

In [6]:
import re

for i in range(len(data_q)):
    data_q[i] = re.sub(r'[^א-ת ]', '', data_q[i]).strip()


In [7]:
from sentence_transformers import SentenceTransformer

# Initialize the embedding model
embedding_fn = SentenceTransformer("paraphrase-multilingual-MiniLM-L12-v2")

# Encode the documents
docs = data_q.copy()
vectors = embedding_fn.encode(docs)  # Encode into vectors
print("Dim:", len(vectors[0]), vectors[0].shape)  # Dim: 768


c:\Users\1\AppData\Local\Programs\Python\Python310\lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Dim: 384 (384,)


In [8]:
data = [
    {"id": i, "vector": vectors[i], "text": docs[i]}
    for i in range(len(vectors))
]
print("Data has", len(data), "entities, each with fields:", data[0].keys())
print("Vector dim:", len(data[0]["vector"]))


Data has 4000 entities, each with fields: dict_keys(['id', 'vector', 'text'])
Vector dim: 384


In [9]:
from pymilvus import FieldSchema, CollectionSchema, DataType, Collection

# Define the collection schema
fields = [
    FieldSchema(name="id", dtype=DataType.INT64, is_primary=True),
    FieldSchema(name="vector", dtype=DataType.FLOAT_VECTOR, dim=len(vectors[0])),
    FieldSchema(name="text", dtype=DataType.VARCHAR, max_length=512),
]
schema = CollectionSchema(fields, description="Demo collection for quotes of Miri Regev")

# Create or load the collection
collection_name = "Miri_Regev"
collection = Collection(name=collection_name, schema=schema)


In [10]:
# Milvus expects the data in columns (field-wise)
field_data = [
    [entity["id"] for entity in data],  # IDs
    [entity["vector"] for entity in data],  # Vectors
    [entity["text"] for entity in data],  # Text
]

# Insert the data
res = collection.insert(field_data)
print(f"Inserted {len(field_data[0])} entities into {collection_name}.")


Inserted 4000 entities into Miri_Regev_quotes.


In [11]:
# Create an index

index_params = {
    "index_type": "IVF_FLAT",
    "metric_type": "L2",
    "params": {"nlist": 128},
}
collection.create_index(field_name="vector", index_params=index_params)
print("Index created.")


Index created.


In [1]:
from pymilvus import list_collections

collections = list_collections()
print("Collections in Milvus:", collections)


ConnectionNotExistException: <ConnectionNotExistException: (code=1, message=should create connection first.)>

In [13]:
collection = Collection(name="Miri_Regev_quotes")


In [14]:
collection.load()

# Define a query vector (e.g., embedding for a new query)
query_vector = embedding_fn.encode(["מה דעתה תחבורה ציבורית?"])[0]

# Perform the search
search_params = {"metric_type": "L2", "params": {"nprobe": 10}}
results = collection.search(
    data=[query_vector],  # Query vector
    anns_field="vector",
    param=search_params,
    limit=15,  # Number of top matches
    output_fields=["text"],  # Include the text field in results
)

# Display results
for result in results:
    for hit in result:
        print(f"Text: {hit.entity.get('text')}")




Text: לתחבורה הציבורית
Text: מי נוסע בתחבורה ציבורית
Text: והתחבורה הציבורית ב
Text: קח תחבורה
Text: עכשיו התפקיד שלנו זה להוסיף את התדירות של האוטובוסים רכבות בעזרת השם  הרכבת מקריית שמונה ועד אילת מתקדמת  רכבות קלות להביא עוד ועוד כלים של תחבורה ציבורית כדי באמת לשכנע את האזרח לוותר על הרכב ולנסוע בתחבורה הציבורית
Text: אתם מעלים את מחיר הנסיעה בתחבורה הציבורית ב
Text: ממשלה שאומרת שאכפת לה מהפריפריה ומהחלשים ובפועל מעלה ב את מחירי הנסיעה בתחבורה הציבורית
Text: יש כאן בעצם ראייה כוללת שמדברת על יכולת של הנגשה וזמינות לאזרח של תחבורה ציבורית  זמינה נגישה וכמובן גם ירוקה עד כמה שניתן
Text: אנחנו רוצים שבכל מתחמי התחנות אדם יגיע מהבית שלו ברכב דוגלגלי יחנה בתחנה ייסע ליעדו על רכבת או על אוטובוס חשמלי או אוטובוס אחר ויחזור
Text: משרד התחבורה הוא משרד גדול משרד שיש לו אחריות אזרחית על אוויר ים ויבשה על כל הכניסה והיציאה אל הארץ ומהארץ החוצה
Text: נדבר על הרכבת הקלה וגם על הכבדה
Text: אני רוצה לתת עוד בשורה שהיא חשובה מאוד ולהודות כאן גם לחברי מקלב סגן השרה שהבאנו איגום משאבים ועצרנו את על

In [15]:
from pymilvus import Collection

# List all collections
from pymilvus import utility
collections = utility.list_collections()
print("Available collections:", collections)

# Load a specific collection
collection_name = "Miri_Regev_quotes"
collection = Collection(name=collection_name)


Available collections: ['Miri_Regev_quotes']
